In [61]:
# run scraper.py to scrap data and save it to review_text_all.txt
import scraper as s
s.runScraper()

In [1]:
import itertools as it
import os
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('en')

## Phrase Modeling

Phrase modeling is an approach to learning combinations of tokens that together represent meaning multi-word concepts.  These phrase models are developed by looping over the words in the corpus and finding words that appear together more than they should by random chance.  The formula used to determine if whether two tokens $A$ and $B$ constitute a phrase is:

$$
\frac {count(A B) - count_{min}}
{count(A) * count(B)}
*N > threshold
$$

where:
* $count(A)$ is the number of times $A$ appears in the corpus
* $count(B)$ is the number of times $B$ appears in the corpus
* $count(AB)$ is the number of times $AB$ appear in the corpus in that order
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that the phrase appears a minimum number of times
* $threshold$ is a user-defined paramter to control how strong the relationship must be before the two tokens are considered a single concept

Once we have trained the phrase model we can apply it to the reviews in our corpus.  It will consider the multiworded tokens to be single phrases.

The gensim library will help us with phrase modeling, specifically the Phrases class.

In [3]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

In [8]:
%mv beerAdvocateScraper/beerAdvocateScraper/BeerAdvocateReviews.csv ./intermediate/

In [10]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.split(',')[1].replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [6]:
intermediate_directory = os.path.join('.', 'intermediate')
review_txt_filepath = os.path.join(intermediate_directory,'BeerAdvocateReviews.csv')
ngram_all = os.path.join(intermediate_directory, 'ngram_all')
unigram_sentences_filepath = os.path.join(ngram_all,
                                          'unigram_sentences_all.txt')

We will use `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text.  We will write this data back out to a new file (`unigram_sentence_all`), with one normalized sentence per line.  We will use this data for learning our phrase models.

In [11]:
with open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        print(sentence)
        f.write(sentence + '\n')

reviewtext
2014 vintage
black and thick
tan head
nose be perfect with huge maple syrup and coffee
bourbon and vanilla
good breakfast ever
taste maple vanilla bourbon and oak
everything -PRON- want in a beer
not overly roasted
just perfect
full bodied
good carbonation
end with maple syrup
amazing
that be all
can not believe -PRON- live up to the hype
12 oz
bottle pour into a snifter
no bottle number
appearance inky black
stain the glass again
big and dark head than mornin delight and assassin
great surface wisps
smell bourbon
maple
vanilla
dark chocolate
everything -PRON- would want in a stout
the bourbon do take over
serve alongside mornin delight and assassin
the beer be near black with a filmy beige head
all hype and bias aside


Our data in the `unigram_sentences_all` file is now organized as a large text file with one sentence per line.  This format allows us to use gensim's LineSentence class, a convenient iterator for working with gensim's other components.  It *streams* the documents/sentences from disk, so you never have ot hold the entire corpus in RAM at once.  This allows you to scale the modeling to a very large corpora.

In [12]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [13]:
for unigram_sentence in it.islice(unigram_sentences, 5,7):
    print(u' '.join(unigram_sentence))
    print(u' ')

bourbon and vanilla
 
good breakfast ever
 


In [14]:
bigram_model_file = os.path.join(ngram_all, 'bigram_model')

In [15]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [16]:
bigram_sentences_filepath = os.path.join(ngram_all,'bigram_sentences_all.txt')

In [17]:
with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for unigram_sentence in unigram_sentences:

        bigram_sentence = u' '.join(bigram_model[unigram_sentence])

        f.write(bigram_sentence + '\n')

In [18]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [19]:
for bigram_sentence in it.islice(bigram_sentences, 5, 7):
    print(u' '.join(bigram_sentence))
    print(u'')

bourbon and vanilla

good breakfast ever



In [20]:
trigram_model_filepath = os.path.join(ngram_all, 'trigram_model')

In [21]:
if 1 == 1:

    phrases_trigram = Phrases(bigram_sentences)
    trigram_model = Phraser(phrases_trigram)
    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

In [22]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [23]:
trigram_sentences_filepath = os.path.join(ngram_all, 'trigram_sentences_all.txt')

In [24]:
with open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for bigram_sentence in bigram_sentences:

        trigram_sentence = u' '.join(trigram_model[bigram_sentence])

        f.write(trigram_sentence + '\n')

In [25]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [26]:
for trigram_sentence in it.islice(trigram_sentences, 3, 5):
    print(u' '.join(trigram_sentence))
    print(u'')

tan head

nose be perfect with huge maple syrup and coffee



In [27]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [28]:
from spacy.lang.en.stop_words import STOP_WORDS

In [29]:

with open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:

    for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                  batch_size=10000, n_threads=4):

        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in STOP_WORDS]
        
        # remove pronouns
        trigram_review = [term for term in trigram_review
                          if term !='-PRON-']
        

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

In [31]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 2, 3):
    print(review)

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 2, 3):
        print(review)

Original:

Black and thick. Tan head. Nose is perfect with huge maple syrup and coffee. Bourbon and vanilla. Best breakfast ever. Taste: maple vanilla bourbon and oak. Everything I want in a beer. Not overly roasted. Just perfect.Full bodied. Good carbonation. Ends with maple syrup.Amazing. That's all. Cannot believe it lives up to the hype.

----

Transformed:

black thick tan head nose perfect huge maple syrup coffee bourbon vanilla good breakfast taste maple vanilla bourbon oak want beer overly roasted perfect bodied good carbonation end maple syrup amazing believe live hype



## Word2Vec

The goal of *word vector embedding models*, or *word vector models* for short is to learn dense numerical representations of each term in a corpus vocabulary.  If the model is succesful, the vectors it learns should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary.  Word vector models are fully unsupervised &mdash; they learn all of these meaning and relationships solely by analyzing the text of the corpus, without any advanced knowledge provided.

In [35]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

In [41]:
if 1==1:
    
    # initiate the model and perform one epoch of training
    beer2vec = Word2Vec(trigram_sentences, size=100, window=5,
                       min_count=1, sg=1) #workers=?
    beer2vec.save(word2vec_filepath)
    
    #perform the next n epochs of training
    for i in range(1, 5):
        beer2vec.train(trigram_sentences, total_examples=beer2vec.corpus_count, epochs=2)
        beer2vec.save(word2vec_filepath)
        
        
#load the finished model from disk
beer2vec = Word2Vec.load(word2vec_filepath)
beer2vec.init_sims()

print(f'Trained model for {beer2vec.train_count} epochs.')

Trained model for 5 epochs.


In [53]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary

len(beer2vec.wv['tan'])


100